## Deploy Ensemble Detector

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import shutil
import os
import glob
import csv
import json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix as confusion_matrix_sklearn
import scipy

from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.data_handling.data_feeding import BatchGenerator
from ketos.neural_networks.resnet import ResNetInterface
from ketos.audio.audio_loader import AudioFrameLoader, AudioLoader, SelectionTableIterator
from ketos.audio.spectrogram import MagSpectrogram
from ketos.neural_networks.dev_utils.detection import batch_load_audio_file_data, filter_by_threshold, filter_by_label, merge_overlapping_detections
from ketos.data_handling.data_feeding import JointBatchGen

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print('done importing packages')

done importing packages


C:\Users\kzammit\Miniconda3\envs\ketos_env\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [2]:
main_folder = r'E:\baseline-with-normalization-reduce-tonal'

#model_names = [main_folder + "\\" + "rs-model-0.kt", main_folder + "\\" + "rs-model-1.kt", main_folder + "\\" + "rs-model-2.kt",
#            main_folder + "\\" + "rs-model-3.kt", main_folder + "\\" + "rs-model-4.kt", main_folder + "\\" + "rs-model-5.kt",
#            main_folder + "\\" + "rs-model-6.kt", main_folder + "\\" + "rs-model-7.kt", main_folder + "\\" + "rs-model-8.kt",
#            main_folder + "\\" + "rs-model-9.kt"]

model_names = [main_folder + "\\" + "rs-model-0.kt", main_folder + "\\" + "rs-model-1.kt"]

spectro_file = r'E:\baseline-with-normalization-reduce-tonal\spec_config_100-1200Hz-0.032-hamm-normalized-reduce-tonal.json'

output_dir = r'E:\baseline-with-normalization-reduce-tonal\deploy\ulu2023\deploy-on-audio\avg'
audio_folder = r'D:\ringed-seal-data\Ulu_2023_St5_Site65\test-subset'

detections_csv = output_dir + '\\' + 'detections-avg.csv'
temp_folder = output_dir + '\\' + 'ringedS_tmp_folder'

In [3]:
# Step 0.5s each time (overlap of 50% for 1 sec duration)
step_size = 0.5

# Number of samples in batch
batch_size = 16

audio_repr = load_audio_representation(path=spectro_file)

spec_config = audio_repr['spectrogram']

audio_loader = AudioFrameLoader(path=audio_folder, duration=spec_config['duration'],
                                step=step_size, stop=False, representation=spec_config['type'],
                                representation_params=spec_config, pad=False)

batch_generator = batch_load_audio_file_data(loader=audio_loader, batch_size=batch_size)

In [4]:
detections_pos = pd.DataFrame()
detections_neg = pd.DataFrame()

for ibx, batch_data in enumerate(batch_generator):

    for idx, model in enumerate(model_names):

        model_name = model.split('\\')[-1].split('.')[0]

        model = ResNetInterface.load(model_file=model, new_model_folder=temp_folder)

        # Run the model on the spectrogram data from the current batch
        batch_predictions = model.run_on_batch(batch_data['data'], return_raw_output=True)

        if idx == 0:
            # Lets store our data in a dictionary
            raw_output_pos = {'filename': batch_data['filename'], 'start': batch_data['start'],
                              'end': batch_data['end'], 'score-pos': batch_predictions[:, 0]}

            raw_output_neg = {'filename': batch_data['filename'], 'start': batch_data['start'],
                              'end': batch_data['end'], 'score-neg': batch_predictions[:, 1]}

        else:
            raw_output_pos |= {'score-pos-' + str(model_name): batch_predictions[:, 0]}
            raw_output_neg |= {'score-neg-' + str(model_name): batch_predictions[:, 1]}

    detections_pos = pd.concat([detections_pos, pd.DataFrame.from_dict(raw_output_pos)])
    detections_neg = pd.concat([detections_neg, pd.DataFrame.from_dict(raw_output_neg)])

  0%|                                                                               | 2/1498 [00:15<2:41:10,  6.46s/it]

100%|██████████████████████████████████████████████████████████████████████████████| 1498/1498 [35:01<00:00,  1.40s/it]


In [5]:
detections_pos

,filename,start,end,score-pos,score-pos-rs-model-1
0,7266.230520000010.wav,0.0,1.0,0.999941,0.999974
1,7266.230520000010.wav,0.5,1.5,0.997628,0.999103
2,7266.230520000010.wav,1.0,2.0,0.758791,0.971695
3,7266.230520000010.wav,1.5,2.5,0.953008,0.996497
4,7266.230520000010.wav,2.0,3.0,0.998166,0.999870
...,...,...,...,...,...
0,7266.230520031510.wav,297.0,298.0,0.998025,0.999906
1,7266.230520031510.wav,297.5,298.5,0.999902,0.999758
2,7266.230520031510.wav,298.0,299.0,0.999631,0.999911
3,7266.230520031510.wav,298.5,299.5,0.999406,0.999740


In [18]:
df_sub = detections_pos[['score-pos', 'score-pos-rs-model-1']].copy()

In [19]:
df_sub['mean'] = df_sub.mean(axis=1)

In [20]:
df_sub

,score-pos,score-pos-rs-model-1,mean
0,0.999941,0.999974,0.999958
1,0.997628,0.999103,0.998365
2,0.758791,0.971695,0.865243
3,0.953008,0.996497,0.974753
4,0.998166,0.999870,0.999018
...,...,...,...
0,0.998025,0.999906,0.998965
1,0.999902,0.999758,0.999830
2,0.999631,0.999911,0.999771
3,0.999406,0.999740,0.999573


In [21]:
detections_pos['score'] = df_sub['mean']

In [22]:
detections_pos

,filename,start,end,score-pos,score-pos-rs-model-1,mean,score
0,7266.230520000010.wav,0.0,1.0,0.999941,0.999974,0.999958,0.999958
1,7266.230520000010.wav,0.5,1.5,0.997628,0.999103,0.998365,0.998365
2,7266.230520000010.wav,1.0,2.0,0.758791,0.971695,0.865243,0.865243
3,7266.230520000010.wav,1.5,2.5,0.953008,0.996497,0.974753,0.974753
4,7266.230520000010.wav,2.0,3.0,0.998166,0.999870,0.999018,0.999018
...,...,...,...,...,...,...,...
0,7266.230520031510.wav,297.0,298.0,0.998025,0.999906,0.998965,0.998965
1,7266.230520031510.wav,297.5,298.5,0.999902,0.999758,0.999830,0.999830
2,7266.230520031510.wav,298.0,299.0,0.999631,0.999911,0.999771,0.999771
3,7266.230520031510.wav,298.5,299.5,0.999406,0.999740,0.999573,0.999573


In [23]:
detections_pos = detections_pos.drop(columns=['score-pos', 'score-pos-rs-model-1', 'mean'])

In [24]:
detections_pos

,filename,start,end,score
0,7266.230520000010.wav,0.0,1.0,0.999958
1,7266.230520000010.wav,0.5,1.5,0.998365
2,7266.230520000010.wav,1.0,2.0,0.865243
3,7266.230520000010.wav,1.5,2.5,0.974753
4,7266.230520000010.wav,2.0,3.0,0.999018
...,...,...,...,...
0,7266.230520031510.wav,297.0,298.0,0.998965
1,7266.230520031510.wav,297.5,298.5,0.999830
2,7266.230520031510.wav,298.0,299.0,0.999771
3,7266.230520031510.wav,298.5,299.5,0.999573


In [25]:
detections_pos['label']=1

In [28]:
detections_pos = detections_pos[detections_pos['score'] >= 0.5]

In [29]:
detections_grp = merge_overlapping_detections(detections_pos)

In [30]:
print(detections_grp)

                  filename  start    end  label     score
0    7266.230520000010.wav    0.0  138.5      1  0.963921
1    7266.230520000010.wav  139.0  147.5      1  0.784610
2    7266.230520000010.wav  148.0  186.0      1  0.998038
3    7266.230520000010.wav  186.5  206.0      1  0.996623
4    7266.230520000010.wav  206.5  255.0      1  0.998865
..                     ...    ...    ...    ...       ...
588  7266.230520031010.wav  299.0  300.0      1  0.999565
589  7266.230520031510.wav    0.0   32.0      1  0.924014
590  7266.230520031510.wav   34.0  141.0      1  0.988741
591  7266.230520031510.wav  141.5  288.0      1  0.997663
592  7266.230520031510.wav  289.5  300.0      1  0.999241

[593 rows x 5 columns]


In [32]:
results_table = detections_grp
audio_folder = r'D:\ringed-seal-data\Ulu_2023_St5_Site65\test-subset'
raven_txt = r'E:\baseline-with-normalization-reduce-tonal\deploy\ulu2023\deploy-on-audio\avg\raven.txt'

cols = ['filename']
results_table.loc[:,cols] = results_table.loc[:,cols].ffill()
results_table['Selection'] = results_table.index +1
results_table['View'] = 'Spectrogram 1'
results_table['Channel'] = 1
results_table['Begin Path'] = audio_folder + '\\' + results_table.filename
results_table['File Offset (s)'] = results_table.start
results_table = results_table.rename(columns={"start": "Begin Time (s)", "end": "End Time (s)", "filename": "Begin File"})
results_table['Begin File'] = results_table['Begin File']
results_table['Low Freq (Hz)'] = 100
results_table['High Freq (Hz)'] = 1200

results_table.to_csv(raven_txt, index=False, sep='\t')